# Learning Supported MPC
## MPC with Gray-box Model of a Bioreactor
## Introduction
This example is meant as a quick guide for solving model predictive control problems with gray-box models. We will see how

- to define a model using Neo modelling framework
- to training an Artificial Neural Network using the Neo-Pythorch wrapper
- to easily construct an gray-box model
- to setup a Nonlinear Model Predictive Control Problem

This example was used in different publications  <cite data-footcite="Ramirez1994">Ramirez et al. (1994)</cite>,
 <cite data-footcite="Tholudur1996">Tholudur et al. (1996)</cite>, <cite data-footcite="Teixeira2006">Teixeira et al. (2006)</cite> and  <cite data-footcite="Morabito2021a">Morabito et al. (2021)</cite>.

<img src="../images/bioreactor.png" alt="drawing" width="600"/>


### Model
In this example we want to control a continuous bioreactor using a Model Predictive Control. The reactor uses _E.Coli_ for the production of $\beta$-galactosidase. The available model that will be used with our MPC is


$$
\begin{align}
    \dot{X} &= \mu X - DX, \\
    \dot{S} &= -R_s X - DS + D_S S_, \\
    \dot{P} &= R_{fp}X - DP, \\
    \dot{I} &= -DI + D_I I_F, \\
\end{align}
$$


where $X,S$ and $I$ are concentrations of biomass, glucose and inducer in g/l respectively, and $P$ is the activity of $\beta$-galactosidase in in enzyme unit per milliliter. The inputs are the glucose diluition rate $D_S$ and inducer diluition rate $D_I$ in 1/h, $S_F$ is the concentration of glucose in the feed $F_S$ and $D = D_S + D_I$. 

The tricky part here is that the kinetic rates $\mu, R_s$ and $R_{fp}$ are unknown. 

### Objective
The goal is maintaining a setpoing $P_{ref}=2$ of $\beta$-galactosidase using an gray-box model where we learned the reaction rates $\mu, R_s$ and $R_{fp}$ from data coming from 5 fedbatch experiments. In this example we will learn the reaction rates using an Artificial Neural Network.

The Model predictive Control problem to solve is

$$
\begin{align}
        &\!\max_{\mathbf{u}(\cdot)}&\qquad& \int_{0}^{T} \Vert(P(t)- P_{ref})\Vert_Q^2 \\
        &\text{s.t.}&&\dot{x}=f(x,\,u,\rho_{w}(x)),\quad x(0)=x_0\\
        &&&x \in [x_{lb}, x_{ub}], \,\ u \in [u_{lb}, u_{ub}].
\end{align}
$$

where the unknown reactions rate are substituted by a neural network $\rho_{w}(x)$. The neural network has 2 features $S$ and $I$ and three labels$\mu, R_s$ and $R_{fp}$.

### Plant model

The model used to simulate the plant is instead more complex and contains two extra states the  inducer shock factor$ISF$, and the inducer recovery factor $IRF$ 


$$
\begin{align}
    \dot{X} &= \mu X - DX, \\
    \dot{S} &= -R_s X - DS + D_S S_, \\
    \dot{P} &= R_{fp}X - DP, \\
    \dot{I} &= -DI + D_I I_F, \\
    \dot{ISF} &= -k_1 ISF\\
    \dot{IRF} &= -k_2 (1- IRF), \\
\end{align}
$$


with the following reaction rates


$$
\begin{align}
    \phi &= \frac{0.4072}{ (0.108 + S + S^2 / 14814.0)}, \nonumber \\
    \mu &= \phi  \left(ISF + \frac{0.22 IRF}{0.22 + I}\right),  \\
    R_s &= 2 \mu, \\
    R_{fp} &= \phi  \frac{0.0005 + I}{0.022 + I}, \\
    k_1 &   = k_2 = \frac{0.09}{0.034 + I}. \nonumber 
\end{align}
$$

## Model Builder
Import necessary libraries

In [1]:
import sys 
from hilo_mpc import Model
import pandas as pd
from hilo_mpc.util.plotting import set_plot_backend
set_plot_backend('bokeh') # Sets the plot backend for all models that are imported from the library

Import the simple model that will be used in the MPC

In [2]:
from hilo_mpc.library.models import  ecoli_D1210_conti
model = ecoli_D1210_conti(model='simple')

## Neural Network Training
The reaction rates are unknown and will be trained using an ANN. First we load the neo-ann

In [3]:
from hilo_mpc import ANN, Layer

then load the data

In [4]:
df = pd.read_csv('../data/learning_ecoli/complete_dataset_5_batches.csv', index_col=0).dropna()
df.head()

,X,S,P,I,V,time,FeedS,FeedI,Sf,If,mu,Rs,Rfp
0.0,0.276405,40.065362,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,100.0,4.0,0.404814,0.809628,0.009200
0.5,0.162451,40.041573,0.195587,0.005323,1.000000,0.5,0.100000,0.050000,100.0,4.0,0.404814,0.809628,0.009200
1.0,0.237256,41.693530,0.000000,0.080970,1.075000,1.0,0.105127,0.052564,100.0,4.0,0.402000,0.804000,0.329145
1.5,0.382575,43.638874,0.000000,0.128871,1.153845,1.5,0.110517,0.055259,100.0,4.0,0.394359,0.788718,0.361241
2.0,0.366413,44.761453,0.000000,0.269108,1.236733,2.0,0.116183,0.058092,100.0,4.0,0.384436,0.768873,0.373405


then train the net

In [5]:
# Neural network features and labels
features = ['S', 'I']  # states of the actual model
labels = ['mu', 'Rs', 'Rfp']  # unknown parameters

# Create and train neural network
ann = ANN(features, labels)
ann.add_layers(Layer.dense(10, activation='sigmoid'))
# ann.add_layers(Layer.dropout(.2))
ann.setup(save_tensorboard=True, tensorboard_log_dir='./runs/ecoli')

# Add the dataset to the trainer
ann.add_data_set(df)

# Train
ann.train(1, 2000, validation_split=.2, patience=100, verbose=1)

Epoch 1/2000
16/16 [==============================] - loss: 0.3891
Evaluate on validation data
4/4 [==============================] - loss: 0.3475
Epoch 2/2000
16/16 [==============================] - loss: 0.2899
Evaluate on validation data
4/4 [==============================] - loss: 0.2542
Epoch 3/2000
16/16 [==============================] - loss: 0.2070
Evaluate on validation data
4/4 [==============================] - loss: 0.1807
Epoch 4/2000
16/16 [==============================] - loss: 0.1435
Evaluate on validation data
4/4 [==============================] - loss: 0.1150
Epoch 5/2000
16/16 [==============================] - loss: 0.0857
Evaluate on validation data
4/4 [==============================] - loss: 0.0623
Epoch 6/2000
16/16 [==============================] - loss: 0.0470
Evaluate on validation data
4/4 [==============================] - loss: 0.0336
Epoch 7/2000
16/16 [==============================] - loss: 0.0279
Evaluate on validation data
4/4 [==================

16/16 [==============================] - loss: 0.0107
Evaluate on validation data
4/4 [==============================] - loss: 0.0050
Epoch 57/2000
16/16 [==============================] - loss: 0.0106
Evaluate on validation data
4/4 [==============================] - loss: 0.0051
Epoch 58/2000
16/16 [==============================] - loss: 0.0104
Evaluate on validation data
4/4 [==============================] - loss: 0.0050
Epoch 59/2000
16/16 [==============================] - loss: 0.0103
Evaluate on validation data
4/4 [==============================] - loss: 0.0051
Epoch 60/2000
16/16 [==============================] - loss: 0.0102
Evaluate on validation data
4/4 [==============================] - loss: 0.0051
Epoch 61/2000
16/16 [==============================] - loss: 0.0098
Evaluate on validation data
4/4 [==============================] - loss: 0.0048
Epoch 62/2000
16/16 [==============================] - loss: 0.0098
Evaluate on validation data
4/4 [=========================

16/16 [==============================] - loss: 0.0045
Evaluate on validation data
4/4 [==============================] - loss: 0.0022
Epoch 113/2000
16/16 [==============================] - loss: 0.0040
Evaluate on validation data
4/4 [==============================] - loss: 0.0027
Epoch 114/2000
16/16 [==============================] - loss: 0.0040
Evaluate on validation data
4/4 [==============================] - loss: 0.0021
Epoch 115/2000
16/16 [==============================] - loss: 0.0042
Evaluate on validation data
4/4 [==============================] - loss: 0.0021
Epoch 116/2000
16/16 [==============================] - loss: 0.0041
Evaluate on validation data
4/4 [==============================] - loss: 0.0024
Epoch 117/2000
16/16 [==============================] - loss: 0.0042
Evaluate on validation data
4/4 [==============================] - loss: 0.0024
Epoch 118/2000
16/16 [==============================] - loss: 0.0037
Evaluate on validation data
4/4 [===================

4/4 [==============================] - loss: 0.0020
Epoch 171/2000
16/16 [==============================] - loss: 0.0027
Evaluate on validation data
4/4 [==============================] - loss: 0.0026
Epoch 172/2000
16/16 [==============================] - loss: 0.0028
Evaluate on validation data
4/4 [==============================] - loss: 0.0022
Epoch 173/2000
16/16 [==============================] - loss: 0.0028
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 174/2000
16/16 [==============================] - loss: 0.0029
Evaluate on validation data
4/4 [==============================] - loss: 0.0027
Epoch 175/2000
16/16 [==============================] - loss: 0.0027
Evaluate on validation data
4/4 [==============================] - loss: 0.0022
Epoch 176/2000
16/16 [==============================] - loss: 0.0028
Evaluate on validation data
4/4 [==============================] - loss: 0.0021
Epoch 177/2000
16/16 [==============================] 

4/4 [==============================] - loss: 0.0025
Epoch 226/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 227/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0024
Epoch 228/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 229/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 230/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 231/2000
16/16 [==============================] - loss: 0.0026
Evaluate on validation data
4/4 [==============================] - loss: 0.0025
Epoch 232/2000
16/16 [==============================] 

## Creating the hybrid model
Now that the NN is created we substitute it to the unknown parameters

In [6]:
model.substitute_from(ann)

we load the "real plant". Note that the real plant has 6 states instead of 4

In [7]:
from hilo_mpc.library.models import  ecoli_D1210_conti
plant = ecoli_D1210_conti(model='complex')
plant.setup(dt=1)
x0_plant = [0.1, 40, 0, 0, 1, 0]
plant.set_initial_conditions(x0_plant)

## Model Predictive Control
We are ready to put everying in the control loop.


In [8]:
from hilo_mpc import NMPC, SimpleControlLoop
model.setup(dt=1)
nmpc = NMPC(model)

nmpc.quad_stage_cost.add_states(names='P', ref=2., weights=10.)
nmpc.quad_terminal_cost.add_states(names='P', ref=2., weights=10.)

nmpc.horizon = 10
nmpc.set_box_constraints(x_lb=[0., 0., 0., 0.], u_lb=[0, 0])
nmpc.set_initial_guess(x_guess=[.1, 40., 0., 0.], u_guess=[0, 0])
nmpc.setup(options={'print_level':0})

We can have print the problem summary as follows

In [9]:
print(nmpc)

             Nonlinear Model Predictive Control                 
-----------------------------------------------------------------
Prediction horizon: 10 
Control horizon: 10 
Model name: ecoli_D1210_conti_simple 
Sampling interval: 1

-----------------------------------------------------------------
Objective function
-----------------------------------------------------------------
Lagrange (stage) cost:
@1=2, ((P-@1)*(10*(P-@1)))
Mayor (terminal) cost:
@1=2, ((P-@1)*(10*(P-@1)))
-----------------------------------------------------------------
Box constraints
-----------------------------------------------------------------
States 
+-------+-----+-----+
| State |  LB |  UB |
+-------+-----+-----+
|   X   | 0.0 | inf |
|   S   | 0.0 | inf |
|   P   | 0.0 | inf |
|   I   | 0.0 | inf |
+-------+-----+-----+
Inputs 
+-------+-----+-----+
| Input |  LB |  UB |
+-------+-----+-----+
|   DS  | 0.0 | inf |
|   DI  | 0.0 | inf |
+-------+-----+-----+
-----------------------------------------

In [10]:
# control_loop = SimpleControlLoop(plant,nmpc)

In [11]:
# control_loop.run(100,p=[100,4])
# control_loop.plot(output_notebook=True)

In [12]:
n_steps = 100
p0 = [100,4]
solution_hybrid = plant.solution
for step in range(n_steps):
    u = nmpc.optimize(x0_plant[0:4], cp=p0)
    plant.simulate(u=u, p = p0)
    x0_plant = solution_hybrid['x:f']


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [13]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import numpy as np

output_notebook()
p_tot = []
for state in plant.dynamical_state_names:
    p = figure(background_fill_color="#fafafa", width=300, height=300)
    p.line(x=np.array(solution_hybrid['t']).squeeze(), y=np.array(solution_hybrid[state]).squeeze(),
           legend_label=state, line_width=2)
    for i in range(len(nmpc.quad_stage_cost._references_list)):
        if state in nmpc.quad_stage_cost._references_list[i]['names']:
            position = nmpc.quad_stage_cost._references_list[i]['names'].index(state)
            value = nmpc.quad_stage_cost._references_list[i]['ref'][position]
            p.line([np.array(solution_hybrid['t'][1]).squeeze(), np.array(solution_hybrid['t'][-1]).squeeze()],
                   [value, value], legend_label=state + '_ref',
                   line_dash='dashed', line_color="red", line_width=2)

    p.yaxis.axis_label = state
    p.xaxis.axis_label = 'time'
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"

    p_tot.append(p)

show(gridplot(p_tot, ncols=2))

Loading BokehJS ...

## Comparing MPC with perfect model

We compare the results of the MPC using the gray-box model with the MPC using the real model.

In [14]:
plant = ecoli_D1210_conti(model='complex')
plant.setup(dt=1)
x0_plant = [0.1, 40, 0, 0, 1, 0]
plant.set_initial_conditions(x0_plant)

# Initialize MPC
nmpc_real = NMPC(plant)

nmpc_real.quad_stage_cost.add_states(names='P', ref=2., weights=10.)
nmpc_real.quad_terminal_cost.add_states(names='P', ref=2., weights=10.)

nmpc_real.horizon = 10
nmpc_real.set_box_constraints(x_lb=[0., 0., 0., 0., 0, 0], u_lb=[0, 0])
nmpc_real.set_initial_guess(x_guess=[.1, 40., 0., 0., 1, 0], u_guess=[0, 0])
nmpc_real.setup(options={'print_level':0})

solution_real = plant.solution
for step in range(n_steps):
    u = nmpc_real.optimize(x0_plant, cp=p0)
    plant.simulate(u=u, p=p0)
    x0_plant = solution_real['x:f']

In [15]:
output_notebook()
p_tot = []
for state in plant.dynamical_state_names:
    p = figure(background_fill_color="#fafafa", width=300, height=300)
    p.line(x=np.array(solution_hybrid['t']).squeeze(), y=np.array(solution_hybrid[state]).squeeze(),
           legend_label=state, line_width=2)
    p.line(x=np.array(solution_real['t']).squeeze(), y=np.array(solution_real[state]).squeeze(),
           legend_label=state + '_real', line_width=2, color='green')
    for i in range(len(nmpc.quad_stage_cost._references_list)):
        if state in nmpc.quad_stage_cost._references_list[i]['names']:
            position = nmpc.quad_stage_cost._references_list[i]['names'].index(state)
            value = nmpc.quad_stage_cost._references_list[i]['ref'][position]
            p.line([np.array(solution_hybrid['t'][1]).squeeze(), np.array(solution_hybrid['t'][-1]).squeeze()],
                   [value, value], legend_label=state + '_ref',
                   line_dash='dashed', line_color="red", line_width=2)

    p.yaxis.axis_label = state
    p.xaxis.axis_label = 'time'
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"

    p_tot.append(p)

show(gridplot(p_tot, ncols=2))

Loading BokehJS ...